## Robot@Home2 - Getting started



`R@H2 notebook series. Version 1.0`



### Introduction



If you have followed previous instructions and you have installed successfully
the robotathome package you are ready to work with it.

Robot@Home2 follows a relational database approach. Instead of a big set of
plain text files, R@H2 stores data in a SQL database. Initially users don't need
to query directly the database, instead the toolbox encapsulates frequent
queries through a set of useful functions.

In this notebook we'll learn the basics to start working on R@H2



### Importing R@H2



In [1]:
import os
import pandas as pd
from robotathome import RobotAtHome
from robotathome import logger, log

In [2]:
log.set_log_level('INFO')  # SUCCESS is the default

level_no, level_name = log.get_current_log_level()
print(f'Current log level name: {level_name}')

Current log level name: INFO


### Instantiating the toolbox



The toolbox is modeled around the RobotAtHome class, so we'll begin instantiating it and
creating a working object.

The RobotAtHome class has some attributes:

-   `rh_path` : string with the full path for robotathome database, usually `rh.db`.
    Its default value is `'.'` (the current directory)
-   `wspc_path` : string with the full path where results or temporary files are
    stored. It's the path to the workspace directory. Its default value also is
    `'.'`.
-   `db_filename` : string with the database filename. Its default value is `'rh.db'`
-   `rgbd_path` : string with the full path to where RGBD images are stored. Its default value is
    `'./files/rgbd'`.
-   `scene_path` : string with the full path to where Scene (3D) images are stored. Its
    default value is `'./files/scene'`.

For this example we have created a `WORKSPACE` directory and we've uncompressed
the dataset under a `R@H2-2.0.1` subdirectory. Therefore our directory structure is as
follows:

    /home/user
    └─── WORKSPACE
         ├─── R@H2-2.0.1
         │    └── files
         │        ├── rgbd
         │        └── scene
         └─────── rh.db

Let's instantiate our first, and usually the only one, object:



In [3]:
my_rh_path = '~/WORKSPACE/R@H2-2.0.1'
my_rgbd_path = os.path.join(my_rh_path, '/files/rgbd')
my_scene_path = os.path.join(my_rh_path, '/files/scene')
my_wspc_path = '~/WORKSPACE'

try: 
    rh = RobotAtHome(my_rh_path, my_rgbd_path, my_scene_path, my_wspc_path)
except:
    logger.error("Something was wrong")

2022-01-22 22:01:21.760 | SUCCESS  | robotathome.core.reader:__open_dataset:87 - Connection is established: rh.db


Now you are ready to dive into the R@H2 database. Following we'll show a set of
functions to extract some of the main reference data.



### Extracting some data



The R@H2 dataset is a collection of raw and processed data from five domestic
settings compiled by a mobile robot Let's start getting the home names:



In [4]:
homes = rh.get_homes()
logger.info("\nHomes: \n{}", homes['name'])

2022-01-22 22:01:21.775 | INFO     | __main__:<module>:2 - 
Homes: 
0      alma
1      anto
2      pare
3       rx2
4    sarmis
Name: name, dtype: object


Notice that the function returns data in a Pandas dataframe. This is the usual
returned data type in R@H2 query functions.

It should be noted that the data set contains one data capture session for all
homes except for *sarmis*. In that case it contains three sessions.

Sessions are characterized by being captures taken in a specific period of
time. Two sessions from the same home may differ, for example, in the
physical arrangement of the objects.

The data capture sessions are:



In [5]:
home_sessions = rh.get_home_sessions()
logger.info("\nSessions: \n{}", home_sessions['name'])

2022-01-22 22:01:21.786 | INFO     | __main__:<module>:2 - 
Sessions: 
0      alma-s1
1      anto-s1
2      pare-s1
3       rx2-s1
4    sarmis-s1
5    sarmis-s2
6    sarmis-s3
Name: name, dtype: object


Furthermore, in the case of *sarmis*, each of the three sessions also contains two
sub-sessions simply labeled `0` and `1`. In that case, the sub-sessions represent
two captures in very close time periods where there has been no there has been
modification of the environment.



In [6]:
# We will call the get_locators function to select the appropriate columns and the
# filtered rows to display sessions and sub-sessions. Don't worry if you get
# confused with this feature. We will come back to it later.
locators = rh.get_locators()[['home_session_name','home_subsession_id']]
logger.info("\nlocators: \n{}", locators[~locators.duplicated()])

2022-01-22 22:01:21.985 | INFO     | __main__:<module>:5 - 
locators: 
   home_session_name  home_subsession_id
0            alma-s1                   0
6            anto-s1                   0
16           pare-s1                   0
27            rx2-s1                   0
32         sarmis-s1                   0
33         sarmis-s1                   1
49         sarmis-s2                   0
50         sarmis-s2                   1
66         sarmis-s3                   0
67         sarmis-s3                   1


There also are a set of typified room classes:



In [7]:
room_types = rh.get_room_types()    # room_types is a Pandas dataframe
# We'll show the dataframe as a list
logger.info("\nRoom types: \n{}", room_types['name'])

2022-01-22 22:01:21.995 | INFO     | __main__:<module>:3 - 
Room types: 
0        bathroom
1         bedroom
2       fullhouse
3         kitchen
4      livingroom
5      masterroom
6        corridor
7    dressingroom
8            hall
Name: name, dtype: object


The rooms have their own explicit names with the prefix of the home to which it
belongs and a suffix that allows it to be uniquely identified in case there is
more than one of the same type.



In [8]:
rooms = rh.get_rooms()
# Change '10' by 'None' to show the full list
pd.set_option('display.max_rows', 10)
logger.info("\nRooms: \n{}", rooms['name'])

2022-01-22 22:01:22.007 | INFO     | __main__:<module>:4 - 
Rooms: 
0         alma_bathroom1
1          alma_bedroom1
2        alma_fullhouse1
3          alma_kitchen1
4       alma_livingroom1
             ...        
36       sarmis_bedroom3
37      sarmis_corridor1
38     sarmis_fullhouse1
39       sarmis_kitchen1
40    sarmis_livingroom1
Name: name, Length: 41, dtype: object


R@H2 data comes from a mobile robot equipped with 4 RGB-D cameras and
a 2D laser scanner. Precisely, those sensor types can be extracted as follows:



In [9]:
sensor_types = rh.get_sensor_types()
logger.info("\nSensor types: \n{}", sensor_types['name'])

2022-01-22 22:01:22.016 | INFO     | __main__:<module>:2 - 
Sensor types: 
0    LASER SCANNER
1      RGBD CAMERA
Name: name, dtype: object


and their names:



In [10]:
sensors = rh.get_sensors()
logger.info("\nSensors: \n{}", sensors['name'])

2022-01-22 22:01:22.025 | INFO     | __main__:<module>:2 - 
Sensors: 
0    HOKUYO1
1     RGBD_1
2     RGBD_2
3     RGBD_3
4     RGBD_4
Name: name, dtype: object


To end this notebook let's try a slightly more complicated query. In this case, the aim is to
obtain a table with the indices and the names of all the combinations of
homes, rooms, sessions, and sub-sessions contained at dataset.



In [11]:
locators = rh.get_locators()
# Change '10' by 'None' to show the full list
pd.set_option('display.max_rows', 10)
# Showing locators columns
logger.info("\nLocators: \n{}", locators.head())

2022-01-22 22:01:22.148 | INFO     | __main__:<module>:5 - 
Locators: 
   home_session_id home_session_name  home_id home_name  room_id  \
0                0           alma-s1        0      alma        0   
1                0           alma-s1        0      alma        1   
2                0           alma-s1        0      alma        2   
3                0           alma-s1        0      alma        3   
4                0           alma-s1        0      alma        4   

          room_name  home_subsession_id  
0    alma_bathroom1                   0  
1     alma_bedroom1                   0  
2   alma_fullhouse1                   0  
3     alma_kitchen1                   0  
4  alma_livingroom1                   0  


Now we can choose the columns to display:



In [12]:
logger.info("\nLocators: \n{}", locators[['home_session_name','home_subsession_id','room_name']])

2022-01-22 22:01:22.157 | INFO     | __main__:<module>:1 - 
Locators: 
   home_session_name  home_subsession_id           room_name
0            alma-s1                   0      alma_bathroom1
1            alma-s1                   0       alma_bedroom1
2            alma-s1                   0     alma_fullhouse1
3            alma-s1                   0       alma_kitchen1
4            alma-s1                   0    alma_livingroom1
..               ...                 ...                 ...
76         sarmis-s3                   0   sarmis_fullhouse1
77         sarmis-s3                   0     sarmis_kitchen1
78         sarmis-s3                   1     sarmis_kitchen1
79         sarmis-s3                   0  sarmis_livingroom1
80         sarmis-s3                   1  sarmis_livingroom1

[81 rows x 3 columns]
